<a href="https://colab.research.google.com/github/jiku100/ShadowDetectionMethod/blob/main/DeepLearning/BDRAR/BDRAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!cp -r "/content/gdrive/My Drive/BDRAR" .

In [3]:
%cd BDRAR

/content/BDRAR


In [ ]:
!unzip SBU-shadow.zip

In [ ]:
import sys
sys.path.append('/content/BDRAR/resnext')
sys.path.append('/content/BDRAR')
print(sys.path)

In [ ]:
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

In [ ]:
import datetime
import os
import datetime

import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from torch.backends import cudnn
from torch.utils.data import DataLoader
from torchvision import transforms
from model import BDRAR
import joint_transforms
from config import sbu_training_root
from dataset import ImageFolder
from misc import AvgMeter, check_mkdir

cudnn.benchmark = True

torch.cuda.set_device(0)

ckpt_path = './ckpt'
exp_name = 'BDRAR'

# batch size of 8 with resolution of 416*416 is exactly OK for the GTX 1080Ti GPU
args = {
    'iter_num': 3000,
    'train_batch_size': 8,
    'last_iter': 0,
    'lr': 5e-3,
    'lr_decay': 0.9,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'snapshot': '',
    'scale': 416
}

joint_transform = joint_transforms.Compose([
    joint_transforms.RandomHorizontallyFlip(),
    joint_transforms.Resize((args['scale'], args['scale']))
])
val_joint_transform = joint_transforms.Compose([
    joint_transforms.Resize((args['scale'], args['scale']))
])
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
target_transform = transforms.ToTensor()
to_pil = transforms.ToPILImage()

train_set = ImageFolder(sbu_training_root, joint_transform, img_transform, target_transform)
train_loader = DataLoader(train_set, batch_size=args['train_batch_size'], num_workers=8, shuffle=True)

bce_logit = nn.BCEWithLogitsLoss().cuda()
log_path = os.path.join(ckpt_path, exp_name, str(datetime.datetime.now()) + '.txt')


def main():
    net = BDRAR().cuda().train()

    optimizer = optim.SGD([
        {'params': [param for name, param in net.named_parameters() if name[-4:] == 'bias'],
         'lr': 2 * args['lr']},
        {'params': [param for name, param in net.named_parameters() if name[-4:] != 'bias'],
         'lr': args['lr'], 'weight_decay': args['weight_decay']}
    ], momentum=args['momentum'])

    if len(args['snapshot']) > 0:
        print('training resumes from \'%s\'' % args['snapshot'])
        net.load_state_dict(torch.load(os.path.join(ckpt_path, exp_name, args['snapshot'] + '.pth')))
        optimizer.load_state_dict(torch.load(os.path.join(ckpt_path, exp_name, args['snapshot'] + '_optim.pth')))
        optimizer.param_groups[0]['lr'] = 2 * args['lr']
        optimizer.param_groups[1]['lr'] = args['lr']

    check_mkdir(ckpt_path)
    check_mkdir(os.path.join(ckpt_path, exp_name))
    open(log_path, 'w').write(str(args) + '\n\n')
    train(net, optimizer)


def train(net, optimizer):
    curr_iter = args['last_iter']
    while True:
        train_loss_record, loss_fuse_record, loss1_h2l_record = AvgMeter(), AvgMeter(), AvgMeter()
        loss2_h2l_record, loss3_h2l_record, loss4_h2l_record = AvgMeter(), AvgMeter(), AvgMeter()
        loss1_l2h_record, loss2_l2h_record, loss3_l2h_record = AvgMeter(), AvgMeter(), AvgMeter()
        loss4_l2h_record = AvgMeter()

        for i, data in enumerate(train_loader):
            optimizer.param_groups[0]['lr'] = 2 * args['lr'] * (1 - float(curr_iter) / args['iter_num']
                                                                ) ** args['lr_decay']
            optimizer.param_groups[1]['lr'] = args['lr'] * (1 - float(curr_iter) / args['iter_num']
                                                            ) ** args['lr_decay']

            with torch.autograd.set_detect_anomaly(True):
                inputs, labels = data
                batch_size = inputs.size(0)
                inputs = Variable(inputs).cuda()
                labels = Variable(labels).cuda()

            

                optimizer.zero_grad()

                fuse_predict, predict1_h2l, predict2_h2l, predict3_h2l, predict4_h2l, \
                predict1_l2h, predict2_l2h, predict3_l2h, predict4_l2h = net.forward(inputs)
                loss_fuse = bce_logit(fuse_predict, labels)
                loss1_h2l = bce_logit(predict1_h2l, labels)
                loss2_h2l = bce_logit(predict2_h2l, labels)
                loss3_h2l = bce_logit(predict3_h2l, labels)
                loss4_h2l = bce_logit(predict4_h2l, labels)
                loss1_l2h = bce_logit(predict1_l2h, labels)
                loss2_l2h = bce_logit(predict2_l2h, labels)
                loss3_l2h = bce_logit(predict3_l2h, labels)
                loss4_l2h = bce_logit(predict4_l2h, labels)

                loss = loss_fuse + loss1_h2l + loss2_h2l + loss3_h2l + loss4_h2l + loss1_l2h + loss2_l2h + loss3_l2h + loss4_l2h
                loss.backward()
                optimizer.step()

                train_loss_record.update(loss.data, batch_size)
                loss_fuse_record.update(loss_fuse.data, batch_size)
                loss1_h2l_record.update(loss1_h2l.data, batch_size)
                loss2_h2l_record.update(loss2_h2l.data, batch_size)
                loss3_h2l_record.update(loss3_h2l.data, batch_size)
                loss4_h2l_record.update(loss4_h2l.data, batch_size)
                loss1_l2h_record.update(loss1_l2h.data, batch_size)
                loss2_l2h_record.update(loss2_l2h.data, batch_size)
                loss3_l2h_record.update(loss3_l2h.data, batch_size)
                loss4_l2h_record.update(loss4_l2h.data, batch_size)

            curr_iter += 1

            log = '[iter %d], [train loss %.5f], [loss_fuse %.5f], [loss1_h2l %.5f], [loss2_h2l %.5f], ' \
                  '[loss3_h2l %.5f], [loss4_h2l %.5f], [loss1_l2h %.5f], [loss2_l2h %.5f], [loss3_l2h %.5f], ' \
                  '[loss4_l2h %.5f], [lr %.13f]' % \
                  (curr_iter, train_loss_record.avg, loss_fuse_record.avg, loss1_h2l_record.avg, loss2_h2l_record.avg,
                   loss3_h2l_record.avg, loss4_h2l_record.avg, loss1_l2h_record.avg, loss2_l2h_record.avg,
                   loss3_l2h_record.avg, loss4_l2h_record.avg, optimizer.param_groups[1]['lr'])
            print(log)
            open(log_path, 'a').write(log + '\n')

            if curr_iter >= args['iter_num']:
                date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                torch.save(net.state_dict(), os.path.join(ckpt_path, exp_name, date + '_%d.pth' % curr_iter))
                return


if __name__ == '__main__':
    main()

In [ ]:
!unzip test.zip

Archive:  test.zip
  inflating: 1.jpg                   
  inflating: 2.jpg                   
  inflating: 3.jpg                   
  inflating: 4.jpg                   
  inflating: 5.jpg                   
  inflating: 6.jpg                   
  inflating: 7.jpg                   
  inflating: Ground1.jpg             
  inflating: Ground10.jpg            
  inflating: Ground11.jpg            
  inflating: Ground12.jpg            
  inflating: Ground13.jpg            
  inflating: Ground14.jpg            
  inflating: Ground2.jpg             
  inflating: Ground3.jpg             
  inflating: Ground4.jpg             
  inflating: Ground5.jpg             
  inflating: Ground6.jpg             
  inflating: Ground7.jpg             
  inflating: Ground8.jpg             
  inflating: Ground9.jpg             


In [ ]:
import numpy as np
import os
import cv2

import torch
from PIL import Image
from torch.autograd import Variable
from torchvision import transforms

from config import sbu_testing_root
from misc import check_mkdir, crf_refine
from model import BDRAR
import matplotlib.pyplot as plt

torch.cuda.set_device(0)

ckpt_path = './ckpt'
exp_name = 'BDRAR'
args = {
    'snapshot': '3000',
    'scale': 416
}

img_transform = transforms.Compose([
    transforms.Resize(args['scale']),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

to_test = {'Ground': "/content/BDRAR"}

to_pil = transforms.ToPILImage()

loadpath = "/content/gdrive/MyDrive/BDRAR/ckpt/BDRAR/2021-03-09 04:01:29_3000.pth"
if not os.path.exists("./Output"):
  os.mkdir("./Output")

def main():
    net = BDRAR().cuda()

    if len(args['snapshot']) > 0:
        print('load snapshot \'%s\' for testing' % args['snapshot'])
        net.load_state_dict(torch.load(loadpath))

    net.eval()
    with torch.no_grad():
        for name, root in to_test.items():
            img_list = [img_name for img_name in os.listdir(root) if
                        img_name.endswith('.jpg')]
            for idx, img_name in enumerate(img_list):
          
                print('predicting for %s: %d / %d' % (name, idx + 1, len(img_list)))
                check_mkdir(
                    os.path.join(ckpt_path, exp_name, '(%s) %s_prediction_%s' % (exp_name, name, args['snapshot'])))
                img = Image.open(os.path.join(root, img_name))
                w, h = img.size
                img_var = Variable(img_transform(img).unsqueeze(0)).cuda()
                res = net(img_var)
                prediction = np.array(transforms.Resize((h, w))(to_pil(res.data.squeeze(0).cpu())))
                prediction = crf_refine(np.array(img.convert('RGB')), prediction)
                Image.fromarray(prediction).save(
                    os.path.join("/content/BDRAR/Output", "Output" + str(idx) + ".jpg"))


if __name__ == '__main__':
    main()

load snapshot '3000' for testing
predicting for Ground: 1 / 21


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


predicting for Ground: 2 / 21
predicting for Ground: 3 / 21
predicting for Ground: 4 / 21
predicting for Ground: 5 / 21
predicting for Ground: 6 / 21
predicting for Ground: 7 / 21
predicting for Ground: 8 / 21
predicting for Ground: 9 / 21
predicting for Ground: 10 / 21
predicting for Ground: 11 / 21
predicting for Ground: 12 / 21
predicting for Ground: 13 / 21
predicting for Ground: 14 / 21
predicting for Ground: 15 / 21
predicting for Ground: 16 / 21
predicting for Ground: 17 / 21
predicting for Ground: 18 / 21
predicting for Ground: 19 / 21
predicting for Ground: 20 / 21
predicting for Ground: 21 / 21


In [ ]:
!zip -r BDRAR_test.zip Output

  adding: Output/ (stored 0%)
  adding: Output/Output20.jpg (deflated 13%)
  adding: Output/Output8.jpg (deflated 2%)
  adding: Output/Output5.jpg (deflated 30%)
  adding: Output/Output14.jpg (deflated 36%)
  adding: Output/Output3.jpg (deflated 10%)
  adding: Output/Output4.jpg (deflated 51%)
  adding: Output/Output19.jpg (deflated 24%)
  adding: Output/Output9.jpg (deflated 4%)
  adding: Output/Output2.jpg (deflated 39%)
  adding: Output/Output6.jpg (deflated 13%)
  adding: Output/Output12.jpg (deflated 20%)
  adding: Output/Output15.jpg (deflated 11%)
  adding: Output/Output11.jpg (deflated 36%)
  adding: Output/Output16.jpg (deflated 18%)
  adding: Output/Output10.jpg (deflated 67%)
  adding: Output/Output7.jpg (deflated 36%)
  adding: Output/Output1.jpg (deflated 43%)
  adding: Output/Output18.jpg (deflated 16%)
  adding: Output/Output17.jpg (deflated 37%)
  adding: Output/Output13.jpg (deflated 48%)
  adding: Output/Output0.jpg (deflated 36%)


In [ ]:
from google.colab import files
files.download('./BDRAR_test.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
if os.path.exists('./ckpt'):
  !cp -r ./ckpt "/content/gdrive/My Drive/BDRAR"